In [4]:
import numpy as np
import pandas as pd
import matplotlib as mlt

In [5]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_e370da00a5a04933b06fe0ef75222851 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='bhpowlmafIZaoyC6CiLY64fFgSbF1j3xommrGCdidJDf',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_e370da00a5a04933b06fe0ef75222851.get_object(Bucket='machinelearningfz-donotdelete-pr-tryhmrnvkimq3u',Key='Black Friday Sales Prediction.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset = pd.read_csv(body)
dataset.head()



Gender  Age  Occupation  Stay in Years  Marital Status  Prod_category  \
0      F   10          10              2               0              3   
1      F   10          10              2               0              1   
2      F   10          10              2               0             12   
3      F   10          10              2               0             12   
4      M   60          16              5               0              8   

   Category  Purchase  
0         1      8370  
1         1     15200  
2         1      1422  
3         1      1057  
4         0      7969

In [6]:
x=dataset.iloc[:,:7].values
x

array([['F', 10, 10, ..., 0, 3, 1],
       ['F', 10, 10, ..., 0, 1, 1],
       ['F', 10, 10, ..., 0, 12, 1],
       ..., 
       ['M', 40, 16, ..., 0, 8, 0],
       ['M', 40, 16, ..., 0, 5, 0],
       ['M', 40, 16, ..., 0, 5, 0]], dtype=object)

In [7]:
y=dataset.iloc[:,7:8].values
y

array([[ 8370],
       [15200],
       [ 1422],
       ..., 
       [ 8043],
       [ 7172],
       [ 6875]])

In [8]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()

In [9]:
x[:,0]=lb.fit_transform(x[:,0])
x

array([[0, 10, 10, ..., 0, 3, 1],
       [0, 10, 10, ..., 0, 1, 1],
       [0, 10, 10, ..., 0, 12, 1],
       ..., 
       [1, 40, 16, ..., 0, 8, 0],
       [1, 40, 16, ..., 0, 5, 0],
       [1, 40, 16, ..., 0, 5, 0]], dtype=object)

In [10]:
from sklearn.model_selection import train_test_split                
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [11]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(x, y)
y_predict = regressor.predict([[1,30,17,1,0,1,0]])
y_predict

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


array([ 15002.81243865])

In [12]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
2019-06-19 07:25:52,739 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


In [13]:
wml_credentials={
    "url": "https://eu-gb.ml.cloud.ibm.com",
    "access_key": "TErMEts-S1OO3e7XdKYnsDAY6srhbCBrN_Bcbp4_s23a",
    "username": "a6898889-068c-4a5d-af3c-2710f950219f",
     "password": "3b34697c-3143-4421-b06b-f4b82f57f6e7",
    "instance_id": "d229f0fe-c06a-44b2-bf5b-bdb870d2a401"
}

In [14]:
client=WatsonMachineLearningAPIClient(wml_credentials)

In [16]:
model_props={
    client.repository.ModelMetaNames.AUTHOR_NAME:"IBM",
    client.repository.ModelMetaNames.AUTHOR_EMAIL:"ibm@ibm.com",
    client.repository.ModelMetaNames.NAME:"Random Forest"
}

In [18]:
model_articraft=client.repository.store_model(regressor,meta_props=model_props)

In [19]:
published_model_uid=client.repository.get_model_uid(model_articraft)

In [20]:
created_deployment=client.deployments.create(published_model_uid,name="Random Forest")



#######################################################################################

Synchronous deployment creation for uid: '842263e0-79ac-476f-9048-484ab647b38a' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='71f1229d-ed42-4f57-a6ec-cfcb5a01b3a8'
------------------------------------------------------------------------------------------------




In [21]:
scoring_endpoint=client.deployments.get_scoring_url(created_deployment)
scoring_endpoint

'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/d229f0fe-c06a-44b2-bf5b-bdb870d2a401/deployments/71f1229d-ed42-4f57-a6ec-cfcb5a01b3a8/online'